In [1]:
%cd /home/ubuntu/RINGFORMER

from __future__ import absolute_import, division, print_function, unicode_literals

import glob
import os
import argparse
import json
import torch
from scipy.io.wavfile import write
from env import AttrDict
from meldataset import mel_spectrogram, MAX_WAV_VALUE, load_wav
from models import Generator
from stft import TorchSTFT

from Utils.JDC.model import JDCNet

/home/ubuntu/miniconda3/envs/respair/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/ubuntu/RINGFORMER


In [2]:
h = None
device = None


def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device)
    print("Complete.")
    return checkpoint_dict


def get_mel(x):
    return mel_spectrogram(x, h.n_fft, h.num_mels, h.sampling_rate, h.hop_size, h.win_size, h.fmin, h.fmax)


def scan_checkpoint(cp_dir, prefix):
    pattern = os.path.join(cp_dir, prefix + '*')
    cp_list = glob.glob(pattern)
    if len(cp_list) == 0:
        return ''
    return sorted(cp_list)[-1]

In [3]:
F0_model = JDCNet(num_class=1, seq_len=192)

In [4]:
cp_path = "/home/ubuntu/RINGFORMER/cp_ringformer_44.1khz"

In [5]:
with open(cp_path + "/config.json") as f:
    data = f.read()

json_config = json.loads(data)
h = AttrDict(json_config)
h

{'F0_path': '/home/ubuntu/Darya_Speech/Utils/JDC/epoch_00079.pth',
 'resblock': '1',
 'num_gpus': 1,
 'batch_size': 18,
 'learning_rate': 0.0002,
 'adam_b1': 0.8,
 'adam_b2': 0.99,
 'lr_decay': 0.999,
 'seed': 1234,
 'upsample_rates': [16, 8],
 'upsample_kernel_sizes': [32, 16],
 'upsample_initial_channel': 512,
 'resblock_kernel_sizes': [3, 7, 11],
 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
 'gen_istft_n_fft': 32,
 'gen_istft_hop_size': 4,
 'segment_size': 65536,
 'num_mels': 128,
 'n_fft': 2048,
 'hop_size': 512,
 'win_size': 2048,
 'sampling_rate': 44100,
 'fmin': 0,
 'fmax': None,
 'fmax_for_loss': None,
 'num_workers': 8,
 'dist_config': {'dist_backend': 'nccl',
  'dist_url': 'tcp://localhost:54321',
  'world_size': 1}}

In [6]:
# device = torch.device('cuda:{:d}'.format(0))

device = 'cuda:0'

In [7]:
generator = Generator(h, F0_model).to(device)
stft = TorchSTFT(filter_length=h.gen_istft_n_fft, hop_length=h.gen_istft_hop_size, win_length=h.gen_istft_n_fft).to(device)

/home/ubuntu/miniconda3/envs/respair/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [8]:
cp_g = scan_checkpoint(cp_path, 'g_')
state_dict_g = load_checkpoint(cp_g, device)
generator.load_state_dict(state_dict_g['generator'])
generator.remove_weight_norm()
_ = generator.eval()

Loading '/home/ubuntu/RINGFORMER/cp_ringformer_44.1khz/g_00017000'


/tmp/ipykernel_3972638/3295719764.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(filepath, map_location=device)


Complete.
Removing weight norm...


### Resynthesis

In [ ]:
import torchaudio
from librosa.filters import mel as librosa_mel_fn
from IPython.display import Audio
import librosa

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=128, n_fft=2048, win_length=2048, hop_length=512, sample_rate=44100, power=2.5)
mean, std = -4, 4

def preprocess(wave):
    wave_tensor = torch.FloatTensor(wav)
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor.to('cuda:0')


wav = librosa.load("/your.wav", sr=44100)[0]

x = preprocess(wav)
print(x.shape)

n = 1
xxx = torch.load("/home/ubuntu/RINGFORMER/gt.pt").to('cuda:0')[n:n+1,:,:]
with torch.no_grad():

    spec, phase = generator(xxx)
    y_g_hat = stft.inverse(spec, phase)
    audio = y_g_hat.squeeze()
    # audio = audio * MAX_WAV_VALUE
    audio = audio.cpu().numpy()


print('Synthesized:')
display(Audio(audio, rate=44100))